In [ ]:
# self-attention
# Query, Key, Value를 이용하여 Attention Score를 계산
# Attention Score를 통해 Weighted Sum을 구함.

import torch
import torch.nn.functional as F 

def self_attention(Q, K, V):
    # Q, K, V: Query, Key, Value 텐서 (입력 벡터)
    # d_k: Query/Key 벡터의 차원 수
    d_k = Q.size(-1)
    
    # Attention Score 계산: Q와 K의 전치행렬을 내적(dot product)하여 계산
    # d_k의 제곱근으로 나누어 스코어의 크기를 조절
    scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(d_k)
    
    # Attention Score를 소프트맥스 함수로 정규화하여 가중치 계산
    # d: 임베딩 차원
    attention_weights = F.softmax(scores, dim=-1)
    
    # 가중치를 이용해 V 벡터의 가중합을 계산하여 최종 출력값 생성
    output = torch.matmul(attention_weights, V)
    
    return output

In [ ]:
# Feed-Foward Neural Network (FFN): 각 단어의 임베딩 벡터를 비선형적으로 변환하는 두 개의 fc layer를 가짐

import torch.nn as nn

# FFN 클래스 정의
class FeedFowardNN(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        
        # 첫 번째 선형 변환층: 입력 차원(d_model)을 중간 차원(d_ff)으로 변환
        self.linear1 = nn.Linear(d_model, d_ff)
        
        # 비선형 활성화 함수 ReLU 정의
        self.relu = nn.ReLU()
        
        # 두 번째 선형 변환층: 중간 차원(d_ff)을 출력 차원(d_model)으로 변환
        self.linear2 = nn.Linear(d_ff, d_model)

    # 순전파 함수 정의
    def forward(self, x):
        # 입력 x에 대해 첫 번째 선형 변환 후 ReLU 활성화 함수 적용,
        # 두 번째 선형 변환을 통해 출력 반환
        return self.linear2(self.relu(self.linear1(x)))

In [ ]:
# Layer Normalization: 각 레이어의 출력을 정규화함.
import torch.nn as nn
import torch

# Layer Normalization 클래스 정의
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        
        # 학습 가능한 파라미터 scale(스케일링)과 shift(이동) 초기화
        # d_model 크기의 1로 초기화된 파라미터 (scale)
        # [1, 1, ... , 1]
        self.scale = nn.Parameter(torch.ones(d_model))
        
        # d_model 크기의 0으로 초기화된 파라미터 (shift)
        # [0, 0, ... , 0]
        self.shift = nn.Parameter(torch.zeros(d_model))
        
        # 작은 값을 더하여 분모가 0이 되는 것을 방지하는 epsilon 설정
        self.eps = eps

    def forward(self, x):
        # 각 마지막 차원(-1)에 대해 평균 계산
        mean = x.mean(-1, keepdim=True)
        
        # 각 마지막 차원(-1)에 대해 표준편차 계산
        std = x.std(-1, keepdim=True)
        
        # 입력 x를 정규화하고 scale과 shift를 적용하여 반환
        return self.scale * (x - mean) / (std + self.eps) + self.shift

In [ ]:
# Residual Connection: 입력에 출력을 더하는 연산

import torch.nn as nn

# 잔차 연결(Residual Connection) 클래스 정의
class ResidualConnection(nn.Module):
    def __init__(self, size):
        super().__init__()
        
        # Layer Normalization 초기화
        self.layer_norm = LayerNorm(size)

    # 순전파 함수 정의
    def forward(self, x, sublayer):
        # 입력 x에 대해 Layer Normalization을 수행한 결과를 sublayer에 전달
        # sublayer의 출력을 원래 입력 x에 더해 반환 (잔차 연결)
        return x + sublayer(self.layer_norm(x))